- augment each pair to N (not every task to 100 (or N*5))
- don't resize
- fit the model on each expanded set of pairs


NEW NEW NEW
- new color definition
- reduced amount of times random method is executed
- class Autoencoder(Model): included input_shape and output_shape
- loooooops

 
TOMORROOOW:
- shapes still don't match up in the model
- ideas:
    * sth to do with the batch???
    * enalrge input to outputsize (resize?)

In [1]:
import numpy as np
import pandas as pd
import random 
from random import randint
import time

import os
import json
from pathlib import Path

import tensorflow.keras as keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import layers
#from tensorflow.keras.datasets import mnist
from tensorflow.keras import backend as K
#from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential

import matplotlib.pyplot as plt
from matplotlib import colors

In [2]:
data_path = Path('./kaggle_input/')
training_path = data_path / 'training'      # 400 X-train ('train'), y-train('test') input-output pairs
evaluation_path = data_path / 'evaluation'  # 400
test_path = data_path / 'test'              # 100 pairs X-train ('train') (input-output),
                                            #           y-train (input only) pairs

training_tasks_files = sorted(os.listdir(training_path))#[1:]
eval_tasks_files = sorted(os.listdir(evaluation_path))
test_task_files = sorted(os.listdir(test_path))

In [3]:
def load_files(path):
    path_files = sorted(os.listdir(path))
    tasks = []
    for task_file in path_files:
        with open(str(path / task_file), 'r') as f:
            task = json.load(f)
            tasks.append(task)
    return tasks

In [4]:
training_tasks = load_files(training_path)          ### this is for building / training the algorithm
test_tasks = load_files(test_path)                  ### this is the input for the actual prediction!!!
evaluation_tasks = load_files(evaluation_path)      ### testing the algorithm

In [113]:
len(training_tasks), training_tasks[0]['test']

(400,
 [{'input': [[7, 0, 7], [7, 0, 7], [7, 7, 0]],
   'output': [[7, 0, 7, 0, 0, 0, 7, 0, 7],
    [7, 0, 7, 0, 0, 0, 7, 0, 7],
    [7, 7, 0, 0, 0, 0, 7, 7, 0],
    [7, 0, 7, 0, 0, 0, 7, 0, 7],
    [7, 0, 7, 0, 0, 0, 7, 0, 7],
    [7, 7, 0, 0, 0, 0, 7, 7, 0],
    [7, 0, 7, 7, 0, 7, 0, 0, 0],
    [7, 0, 7, 7, 0, 7, 0, 0, 0],
    [7, 7, 0, 7, 7, 0, 0, 0, 0]]}])

In [111]:
len(evaluation_tasks), evaluation_tasks[0]['test']

(400,
 [{'input': [[3, 2], [7, 8]],
   'output': [[3, 2, 3, 2, 3, 2],
    [7, 8, 7, 8, 7, 8],
    [2, 3, 2, 3, 2, 3],
    [8, 7, 8, 7, 8, 7],
    [3, 2, 3, 2, 3, 2],
    [7, 8, 7, 8, 7, 8]]}])

In [112]:
len(test_tasks),test_tasks[0]['test']

(100, [{'input': [[3, 2], [7, 8]]}])

 
 ---
 ---

### 1. Create Train - Test - Split

In [5]:
def tts(input_list):
    Xs_test, ys_test, Xs_train, ys_train = [], [], [], []

    for task in input_list:
        X_test, y_test, X_train, y_train = [], [], [], []

        for pair in task["test"]:
            X_test.append(pair["input"])
            y_test.append(pair["output"])      ### to be predicted !!!

        for pair in task["train"]:
            X_train.append(pair["input"])
            y_train.append(pair["output"])

        Xs_test.append(X_test)
        ys_test.append(y_test)
        Xs_train.append(X_train)
        ys_train.append(y_train)
    return Xs_test, ys_test, Xs_train, ys_train

 
#### Create origanl sized Train-Test-Split for reference 

In [6]:
Xs_test_orig, ys_test_orig, Xs_train_orig, ys_train_orig = tts(training_tasks)

In [7]:
len(Xs_train_orig), len(ys_train_orig), len(Xs_test_orig), len(ys_test_orig)

(400, 400, 400, 400)

In [8]:
len(Xs_train_orig[0]), len(ys_train_orig[0]), len(Xs_test_orig[0]), len(ys_test_orig[0])

(5, 5, 1, 1)

 
#### Create actual Train-Test-Split


In [9]:
Xs_test, ys_test, Xs_train, ys_train = tts(training_tasks)

In [10]:
len(Xs_train), len(ys_train), len(Xs_test), len(ys_test)

(400, 400, 400, 400)

In [11]:
len(Xs_train[0]), len(ys_train[0]), len(Xs_test[0]), len(ys_test[0])

(5, 5, 1, 1)

## Data Augmentation of the training data

In [12]:
#### color
def color_change(arr):
    x = arr*3
#     x = x/(x.max()+0.1)
    x = np.interp(x, (0.3, 2.7), (0, 0.9))
    x = np.ndarray.round(x,1)
    return x

In [13]:
def augm(X_list, y_list):
    start = time.time()
    idx = 1
    X_list_new = []
    y_list_new = []
    
    for task_X, task_y in zip(X_list, y_list):
        print(len(task_X))
        task_x_li = []
        task_y_li = []
        
        for e_x, e_y in zip(task_X, task_y):
            
            new_x = np.array(e_x)     
            e_x_li = []
            e_x_li.append(new_x.reshape(new_x.shape + (1,) ))
            
            new_y = np.array(e_y)
            e_y_li = []
            e_y_li.append(new_y.reshape(new_y.shape + (1,) ))
            
            while len(e_x_li) < 10:            ###############
                """
                choose a random method and append new array
                """
                times = randint(1,4)
                for j in range(times):

                    method = random.choice([np.fliplr, np.flipud, np.rot90,color_change])
                    new_x = method(new_x)
                    new_y = method(new_y)

                e_x_li.append(new_x.reshape(new_x.shape + (1,) ))
                e_y_li.append(new_y.reshape(new_y.shape + (1,) ))
                
            task_x_li.append(e_x_li)
            task_y_li.append(e_y_li)
            
        X_list_new.append(task_x_li)
        y_list_new.append(task_y_li)
            
#             idx +=1
    end = time.time()
 
    print("Total time: " + str(np.round(end - start, 1)) + " s" + "\n")
    return X_list_new, y_list_new

In [14]:
X_train_aug, y_train_aug = augm(Xs_train, ys_train)

5
5
3
2
3
3
3
3
3
2
4
2
4
3
3
4
3
3
4
3
3
3
3
3
3
5
3
2
3
3
3
3
3
4
3
2
3
3
2
3
3
3
3
3
3
4
2
6
5
8
3
4
4
3
2
7
3
5
3
2
4
4
3
4
3
3
3
3
2
3
2
4
3
4
3
3
3
3
3
3
2
2
3
3
2
3
4
4
4
4
3
2
3
2
2
3
4
3
3
3
3
4
6
2
3
3
3
3
3
3
3
3
2
3
3
4
3
4
3
3
3
3
3
3
2
3
4
3
3
2
3
4
4
3
4
3
3
3
2
2
3
3
3
4
4
4
4
3
4
3
3
4
3
3
3
2
2
3
3
2
3
3
4
4
3
3
5
2
3
3
4
4
3
3
3
3
3
5
4
5
3
4
3
3
4
10
3
3
3
3
4
2
3
3
3
3
4
3
3
3
4
4
4
5
3
3
3
3
3
3
3
2
3
3
2
3
3
3
3
4
4
3
2
3
3
3
4
3
4
3
3
3
3
3
4
4
4
3
4
3
3
3
3
3
4
3
6
3
3
2
3
3
2
2
3
3
6
3
3
3
3
4
4
2
3
4
2
3
3
3
4
4
3
6
3
3
2
2
4
4
3
2
2
3
3
2
4
4
5
3
3
5
4
2
5
5
3
3
2
4
3
3
3
3
3
2
3
3
3
3
3
2
3
3
3
3
2
4
4
3
5
3
2
3
3
3
3
4
3
3
2
4
3
7
3
2
3
3
4
3
3
3
3
3
2
4
5
2
4
4
3
3
7
3
4
3
3
2
3
3
3
4
3
3
3
3
3
2
3
3
2
3
2
4
3
2
3
4
3
4
3
3
3
3
2
5
4
3
3
3
4
3
3
3
5
3
3
5
8
3
Total time: 0.9 s



In [15]:
type(X_train_aug),len(X_train_aug)   ### number of Tasks

(list, 400)

In [16]:
type(X_train_aug[0]),len(X_train_aug[0]) ### number of sets

(list, 5)

In [17]:
type(X_train_aug[0][0]),len(X_train_aug[0][0]) ### number of inputs (pairs)

(list, 10)

In [22]:
X_train_aug[0][0][0]

array([[[0],
        [7],
        [7]],

       [[7],
        [7],
        [7]],

       [[0],
        [7],
        [7]]])

In [25]:
np.array(X_train_aug[0][0]).shape

(10, 3, 3, 1)

#### check format of all train test splits

In [ ]:
type(Xs_train_tasks), type(ys_train_tasks), type(Xs_test_tasks), type(ys_test_tasks)

In [ ]:
len(Xs_train_tasks), len(ys_train_tasks), len(Xs_test_tasks), len(ys_test_tasks)

In [ ]:
type(Xs_train_tasks[0]), type(ys_train_tasks[0]),type(Xs_test_tasks[0]), type(ys_test_tasks[0])

In [ ]:
len(Xs_train_tasks[0]), len(ys_train_tasks[0]),len(Xs_test_tasks[0]), len(ys_test_tasks[0])

In [ ]:
Xs_train_tasks[0][0].shape, ys_train_tasks[0][0].shape,Xs_test_tasks[0][0].shape, ys_test_tasks[0][0].shape

In [26]:
def t2l(split_list):
    li =[]
    for task in split_list:
        task_arr = np.array(task)
        li.append(task_arr)
    return li

In [29]:
Xs_train_tasks = t2l(X_train_aug)
ys_train_tasks = t2l(y_train_aug)
Xs_test_tasks = t2l(Xs_test)
ys_test_tasks = t2l(ys_test)

/opt/anaconda3/envs/tf/lib/python3.6/site-packages/ipykernel_launcher.py:4: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  after removing the cwd from sys.path.


In [31]:
type(Xs_train_tasks), type(ys_train_tasks),type(Xs_test_tasks), type(ys_test_tasks)

(list, list, list, list)

In [30]:
type(Xs_train_tasks[0]), type(ys_train_tasks[0]),type(Xs_test_tasks[0]), type(ys_test_tasks[0])

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [32]:
type(Xs_train_tasks[0][0]), type(ys_train_tasks[0][0]),type(Xs_test_tasks[0][0]), type(ys_test_tasks[0][0])

(numpy.ndarray, numpy.ndarray, numpy.ndarray, numpy.ndarray)

In [47]:
type(Xs_train_tasks[0][0][0]),Xs_train_tasks[0][0][0]

(numpy.ndarray,
 array([[[0.],
         [7.],
         [7.]],
 
        [[7.],
         [7.],
         [7.]],
 
        [[0.],
         [7.],
         [7.]]]))

In [45]:
type(Xs_train_tasks[0][0][0][0]),Xs_train_tasks[0][0][0][0]

(numpy.ndarray,
 array([[0.],
        [7.],
        [7.]]))

In [44]:
type(Xs_train_tasks[0][0][0][0][0]),Xs_train_tasks[0][0][0][0][0]

(numpy.ndarray, array([0.]))

In [39]:
type(Xs_train_tasks[0][0][0][0][0][0]),Xs_train_tasks[0][0][0][0][1][0]

numpy.float64

In [43]:
Xs_train_tasks[0][0][0][0][1][0]

7.0

## Model comp

In [93]:
class Autoencoder(Model):
    def __init__(self, latent_dim, inp_shape, outp_shape, flat_shape):
        super(Autoencoder, self).__init__()
        self.latent_dim = latent_dim
        self.inp_shape = inp_shape
        self.outp_shape = outp_shape
        self.flat_shape = flat_shape
        
        
        self.encoder = Sequential([
            layers.Conv2D(160, (3, 3), activation='relu', padding='same', input_shape = inp_shape), #### 
            layers.MaxPooling2D((2, 2), padding='same'),
            layers.Flatten(),
            layers.Dense(latent_dim, activation='relu'),
        ])
        self.decoder = Sequential([
#             layers.InputLayer(input_shape=(latent_dim,)),
            layers.Dense(flat_shape, activation='sigmoid'),
            layers.Reshape(outp_shape)
        ])

    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

In [94]:
K.clear_session()

In [96]:
K.clear_session()   #clean slate
latent_dim = 64 
idx = 0
#start = time.time()

model = Sequential()

for X_train, y_train, X_test, y_test in zip(Xs_train_tasks, ys_train_tasks,Xs_test_tasks, ys_test_tasks):    
    print("TASK " + str(idx + 1))
    for X_set_train, y_set_train, X_set_test, y_set_test in zip(X_train, y_train, X_test, y_test):
        
        ### get input and output shapes to pass to the model
        inp_shape = X_set_train[0].shape
        print(inp_shape)
        outp_shape = y_set_train[0].shape#[0]*y_set_train[0].shape[1]
        print('out',outp_shape)
        flat_shape = outp_shape[0]*outp_shape[1]
        print('flat',flat_shape)
        
        base_model = Autoencoder(latent_dim, inp_shape, outp_shape, flat_shape)

        base_model.compile(optimizer='adam', 
                              loss='mean_squared_error',
                              metrics=['accuracy'])

        callback = keras.callbacks.EarlyStopping(monitor='accuracy',
                                                 patience=5)

        if idx == 0 or idx == 398:
            base_model.fit(X_set_train, y_set_train,
                          epochs=100,
                          batch_size=5000,
                          #shuffle=True,
    #                       verbose=0,
                          callbacks=[callback],
                          validation_data=(X_set_test, y_set_test))


        else:
            base_model.fit(X_set_train, y_set_train,
                          epochs=100,
                          batch_size=5000,
                          #shuffle=True,
                          callbacks=[callback],
                          verbose=0,
                          validation_data=(X_set_test, y_set_test))


    idx += 1

TASK 1
(3, 3, 1)
out (9, 9, 1)
flat 81
Epoch 1/100
1/1 [==============================] - ETA: 0s - loss: 2.7429 - accuracy: 0.1728

ValueError: Data cardinality is ambiguous:
  x sizes: 3
  y sizes: 9
Make sure all arrays contain the same number of samples.

## MAKE PREDICTIONS

In [ ]:
encoded_imgs = base_model.encoder(Xs_test_tasks[125]).numpy()
decoded_imgs = base_model.decoder(encoded_imgs).numpy()

In [ ]:
decoded_imgs.shape

In [ ]:
w = Xs_test_tasks[125:126][0]#.shape

In [ ]:
w.shape

In [ ]:
v = w*10

In [ ]:
y = ys_test_tasks[125].reshape(32,32)

In [ ]:
y.shape

In [ ]:
y=y*10

In [ ]:
y

In [ ]:
base_model.predict(w)#.shape

In [ ]:
pred = base_model.predict(w).reshape(1,32,32)

In [ ]:
pred.shape

In [ ]:
arr=(decoded_imgs[0].reshape(32,32))*10

In [ ]:
new_arr = np.around(pred,1)*10

In [ ]:
cmap = colors.ListedColormap(
            ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
             '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, ax = plt.subplots(1, 4, figsize=(15,15))
ax[0].imshow(v[0], cmap=cmap, norm=norm)
ax[0].set_title('Train Input')
ax[1].imshow(y, cmap=cmap, norm=norm)
ax[1].set_title('Train Output')
ax[2].imshow(new_arr[0], cmap=cmap, norm=norm)
ax[2].set_title('Train Prediction')
ax[3].imshow(arr, cmap=cmap, norm=norm)
ax[3].set_title('Train Prediction2');

In [ ]:
i = 0
c = 0
for task in Xs_train_orig:
    i+=1
    j = 0
    for e in task:
        j+=1
        c += 1
        if c > 124:
            break
    if c > 124:
        break

In [ ]:
i,j

In [ ]:
 ww = (np.array(Xs_train_tasks[40][0]).reshape(32,32))*10

In [ ]:
ww

In [ ]:
vv = (np.array(ys_train_tasks[40][0]).reshape(32,32))*10

In [ ]:
tt = ww.reshape(1,32,32,1)

In [ ]:
encoded_imgs = base_model.encoder(tt).numpy()
decoded_imgs = base_model.decoder(encoded_imgs).numpy()

In [ ]:
arr=(decoded_imgs[0].reshape(32,32))*10

In [ ]:
arr[:10,:10].shape

In [ ]:
np.asarray(Xs_train_orig[40][0]).shape, np.asarray(ys_train_orig[40][0]).shape

In [ ]:
cmap = colors.ListedColormap(
            ['#000000', '#0074D9','#FF4136','#2ECC40','#FFDC00',
             '#AAAAAA', '#F012BE', '#FF851B', '#7FDBFF', '#870C25'])
norm = colors.Normalize(vmin=0, vmax=9)
fig, ax = plt.subplots(1, 4, figsize=(15,15))
ax[0].imshow(ww[:10,:10], cmap=cmap, norm=norm)
ax[0].set_title('Train Input')
ax[1].imshow(vv[:10,:10], cmap=cmap, norm=norm)
ax[1].set_title('Train Output')
ax[2].imshow(new_arr[0][:10,:10], cmap=cmap, norm=norm)
ax[2].set_title('Train Prediction')
ax[3].imshow(arr[:10,:10], cmap=cmap, norm=norm)
ax[3].set_title('Train Prediction2');

# ---------------------------------------------------------------------------------------------

In [ ]:
def prep_tasks(split_list):
    split_tasks = []
    for task in split_list:
        li=[]
        for e in task:
            e = np.array(e)
            e = np.pad(e, [(0, 32-e.shape[0]), (0, 32-e.shape[1])], mode='constant')
            e = e / 10.
            e = e.reshape(e.shape + (1,) )
            li.append(e)
#         li = np.array(li)
        split_tasks.append(li)
    return split_tasks

In [ ]:
Xs_train_tasks = prep_tasks(Xs_train)
Xs_test_tasks = prep_tasks(Xs_test)
ys_train_tasks = prep_tasks(ys_train)
ys_test_tasks = prep_tasks(ys_test)



In [ ]:
len(Xs_train), len(ys_train), len(Xs_test), len(ys_test)

In [ ]:
len(Xs_train[0]), len(ys_train[0]), len(Xs_test[0]), len(ys_test[0])